In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import json
from cryptocompy import coin,price
import tqdm
import seaborn as sns
sns.set();

import io
import requests
import json
from datetime import datetime
import time
from time import gmtime, strftime

from sklearn import metrics
from fbprophet import Prophet
plt.rcParams["figure.figsize"] = [14,8]
pd.set_option('float_format', '{:f}'.format)

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import datetime
from datetime import datetime as dt
from datetime import timedelta


from tqdm import tqdm_notebook
from livelossplot.keras import PlotLossesCallback


from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
#from xgboost import XGBRegressor
from catboost import CatBoostRegressor, Pool, cv
import catboost
from sklearn.ensemble import RandomForestRegressor

from keras.models import Sequential
from keras.layers import Dense
from keras.models import model_from_json
#from sklearn.cross_validation import cross_val_score
#from sklearn.grid_search import GridSearchCV
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(1)

from keras.models import Sequential  
from keras.layers.core import Dense, Activation  
from keras.layers.recurrent import LSTM
from keras.layers.advanced_activations import LeakyReLU

from keras import optimizers
from keras import backend as K

adam = optimizers.Adam(lr=0.01, clipnorm=1.)

def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 
def nn_r2_score(y_true, y_pred):
    total_error = K.sum(K.square( y_true - K.mean(y_true) ) )
    residual_error = K.sum(K.square( y_true - y_pred ))
    R_squared = 1 - (residual_error / total_error)
    return -R_squared


from math import sin, cos

from gaft import GAEngine
from gaft.components import BinaryIndividual,DecimalIndividual
from gaft.components import Population
from gaft.operators import TournamentSelection, LinearRankingSelection
from gaft.operators import UniformCrossover
from gaft.operators import FlipBitMutation
from gaft.analysis import ConsoleOutput

# Analysis plugin base class.
from gaft.plugin_interfaces.analysis import OnTheFlyAnalysis

# Built-in best fitness analysis.
from gaft.analysis.fitness_store import FitnessStore

import MySQLdb 
import sshtunnel
from sshtunnel import SSHTunnelForwarder

Using TensorFlow backend.


In [3]:
df = pd.DataFrame()
d1 = pd.DataFrame()
d2 = pd.DataFrame()
d3 = pd.DataFrame()
with SSHTunnelForwarder(('206.189.186.74', 22), ssh_password='crypto1234', ssh_username='aagam', remote_bind_address=('127.0.0.1', 3306)) as server:
    conn = MySQLdb.connect(host='localhost', port=server.local_bind_port, user='ianbalina', passwd='5!sT3jt26K%tFN*W',db='ianbalina')
    cursor = conn.cursor()
    
    #cursor.execute("SELECT * FROM ianbalina.ico_price_daily_summaries;") 
    #m = cursor.fetchone()
    query = '''SELECT * FROM ianbalina.ico_price_daily_summaries;'''
    data = pd.read_sql_query(query, conn)
    df = pd.concat([df,data])
    
    
    query = '''SELECT * FROM ianbalina.ico_grade_history;'''
    data = pd.read_sql_query(query, conn)
    d1 = pd.concat([d1,data])
    
    query = '''SELECT * FROM ianbalina.ico_analytics;'''
    data = pd.read_sql_query(query, conn)
    d2 = pd.concat([d2,data])
    
    query = '''SELECT * FROM ianbalina.icos;'''
    data = pd.read_sql_query(query, conn)
    d3 = pd.concat([d3,data])
    
    
    conn.close() 

I1004 08:04:43.449102 10732 transport.py:1819] Connected (version 2.0, client OpenSSH_7.6p1)
I1004 08:04:46.834755 10732 transport.py:1819] Authentication (password) successful!


In [18]:
d1['date'].unique()

array([datetime.date(2019, 10, 4)], dtype=object)

In [17]:
d1 = d1[d1['date'] == d1['date'].sort_values().iloc[-1]]

In [15]:
ids = d3['id'][d3['published_at'].notnull()].unique().tolist()

In [37]:
coin = []
for id in ids:
    coin.append(df['ico_symbol'][df['ico_id'] == id].unique().tolist()[0])

In [38]:
temp = pd.DataFrame()
temp['Coin'] = coin
temp['Score'] = d1['tokenmetrics_score'][d1['ico_id'].isin(ids)].values

temp = pd.DataFrame()
#d1 = d1[d1['date'] == datetime.date(2019, 9, 19)]
for coin in df['ico_symbol'].unique().tolist():
    num = df['ico_id'][df['ico_symbol'] == coin].unique().tolist()
    t = d1['tokenmetrics_score'][d1['ico_id'].isin(num)]
    if len(t)>0:
        t = t.iloc[-1]
        temp = pd.concat([temp, pd.DataFrame([coin,t]).T])

In [41]:
t2 = []
coins = []
for coin in temp['Coin'].tolist():
    t1 = df['ico_id'][df['ico_symbol'] == coin].unique().tolist()
    x = d2['all_time_growth_rate'][d2['ico_id'].isin(t1)].dropna().unique().tolist()
    if len(x) != 0:
        t2.append(x[-1])
        coins.append(coin)

In [42]:
t2 = pd.DataFrame([coins,t2])
t2 = t2.T
t2.columns = ['Coin','ROI']
temp.columns = ['Coin','Score']

In [43]:
temp = pd.merge(temp,t2)

In [45]:
temp[['Score','ROI']] = temp[['Score','ROI']].astype('float64')

In [48]:
'Correlation: ', temp.corr().iloc[0,1]

('Correlation: ', 0.35571144248837555)

In [49]:
temp['Score_bins'] = pd.cut(temp['Score'],[0,10,20,30,40,50,60,70,80,90,100])
temp['ROI_bins'] = pd.cut(temp['ROI'],[-100,0,6000000])

In [84]:
temp

,Coin,Score,ROI,Score_bins,ROI_bins
0,TRX,63.429900,647.239500,"(60, 70]","(0, 6000000]"
1,ONT,67.164900,200.850100,"(60, 70]","(0, 6000000]"
2,ETH,96.634000,6080.551800,"(90, 100]","(0, 6000000]"
3,NEO,79.581700,3579.078100,"(70, 80]","(0, 6000000]"
4,BNB,80.081700,13516.003900,"(80, 90]","(0, 6000000]"
5,ADA,74.160000,1479.738200,"(70, 80]","(0, 6000000]"
6,LSK,73.702400,1031.416900,"(70, 80]","(0, 6000000]"
7,REP,75.340000,1288.814800,"(70, 80]","(0, 6000000]"
8,QTUM,71.544900,461.292900,"(70, 80]","(0, 6000000]"
9,OMG,83.665000,53.214800,"(80, 90]","(0, 6000000]"


In [56]:
final = temp['ROI_bins'].groupby([temp['Score_bins'],temp['ROI_bins']]).count().unstack()
final

ROI_bins    (-100, 0]  (0, 6000000]
Score_bins                         
(40, 50]     1.000000      1.000000
(50, 60]          nan      1.000000
(60, 70]     2.000000      5.000000
(70, 80]     4.000000      9.000000
(80, 90]     2.000000      4.000000
(90, 100]         nan      2.000000

In [94]:
[0,0,0,0] + final.values[:,0].tolist()

[0, 0, 0, 0, 1.0, nan, 2.0, 4.0, 2.0, nan]

In [99]:
x = pd.DataFrame()
x['(-100, 0]'] = [0,0,0,0] + final.values[:,0].tolist()
x['(0, 6000000]'] = [0,0,0,0] + final.values[:,1].tolist()
x.index = ['(0,10]','(10,20]','(20,30]','(30,40]','(40,50]','(50,60]','(60,70]','(70,80]','(80,90]','(90,100]']
x['Accuracy'] = [0,0,0,0,50,100, 5/7*100, 9/13*100, 4/6*100, 100]

x.fillna(0,inplace=True)

In [101]:
x

,"(-100, 0]","(0, 6000000]",Accuracy
"(0,10]",0.000000,0.000000,0.000000
"(10,20]",0.000000,0.000000,0.000000
"(20,30]",0.000000,0.000000,0.000000
"(30,40]",0.000000,0.000000,0.000000
"(40,50]",1.000000,1.000000,50.000000
"(50,60]",0.000000,1.000000,100.000000
"(60,70]",2.000000,5.000000,71.428571
"(70,80]",4.000000,9.000000,69.230769
"(80,90]",2.000000,4.000000,66.666667
"(90,100]",0.000000,2.000000,100.000000


In [71]:
x.columns = ['(-100, 0]']

In [72]:
x['(0, 6000000]'] = [0.0,0.0,0.0,0.0]

In [73]:
x.dtypes

(-100, 0]       float64
(0, 6000000]    float64
dtype: object

In [76]:
final.index

CategoricalIndex([(40, 50], (50, 60], (60, 70], (70, 80], (80, 90], (90, 100]], categories=[(0, 10], (10, 20], (20, 30], (30, 40], (40, 50], (50, 60], (60, 70], (70, 80], ...], ordered=True, name='Score_bins', dtype='category')

In [74]:
final.dtypes

ROI_bins
(-100, 0]       float64
(0, 6000000]    float64
dtype: object

In [75]:
pd.concat([final,x])

TypeError: '<' not supported between instances of 'int' and 'str'

In [57]:
pd.DataFrame(final)

ROI_bins    (-100, 0]  (0, 6000000]
Score_bins                         
(40, 50]     1.000000      1.000000
(50, 60]          nan      1.000000
(60, 70]     2.000000      5.000000
(70, 80]     4.000000      9.000000
(80, 90]     2.000000      4.000000
(90, 100]         nan      2.000000

In [51]:
temp['ROI_bins'].groupby([temp['Score_bins'],temp['ROI_bins']]).count()

Score_bins  ROI_bins    
(40, 50]    (-100, 0]       1
            (0, 6000000]    1
(50, 60]    (0, 6000000]    1
(60, 70]    (-100, 0]       2
            (0, 6000000]    5
(70, 80]    (-100, 0]       4
            (0, 6000000]    9
(80, 90]    (-100, 0]       2
            (0, 6000000]    4
(90, 100]   (0, 6000000]    2
Name: ROI_bins, dtype: int64

In [15]:
t1 = temp[['Coin','Score','ROI']][temp['ROI'] <= 0]
t1[t1['Score'] > 50]

,Coin,Score,ROI
6,XMR,79.226000,-15.416100
7,DASH,76.990000,-41.958200
9,ZEC,77.905000,-54.415600
20,BCH,71.415000,-26.728200
23,ETC,74.794000,-22.949900
58,THETA,74.970500,-28.783100
113,XEM,67.495000,-41.439500
115,DCR,80.980500,-26.650600
116,NANO,61.258900,-61.128200
172,STX,79.645900,-89.084400


In [16]:
t1[t1['Score'] <= 50]

,Coin,Score,ROI
2,AVT,3.527000,-92.809600
5,USDT,26.162200,-0.218800
10,BTG,40.330000,-55.536500
11,SNT,10.795000,-46.527300
13,SAN,9.383500,-64.127500
14,FUN,6.679100,-58.176200
15,TNB,14.805500,-84.090300
22,ELF,17.249000,-31.424900
25,CND,12.253400,-57.118400
26,DRGN,8.024800,-46.831600


In [17]:
t2 = temp[['Coin','Score','ROI']][temp['ROI'] > 0]

In [18]:
t2[t2['Score'] > 50]

,Coin,Score,ROI
0,BTC,91.126000,6119.023400
1,ETH,97.500000,52970.218800
3,OMG,75.165000,122.764700
4,EOS,72.164700,169.462700
8,NEO,79.581700,3474.327900
16,BAT,86.610200,360.401300
17,TRX,63.429900,563.101600
18,HOT,75.633900,329.160000
21,XRP,71.598800,5847.043000
24,QTUM,71.544800,403.675600


In [19]:
t2[t2['Score'] <= 50]

,Coin,Score,ROI
12,ETP,36.564900,6.781600
19,GNT,17.340000,377.465900
41,ZIL,48.548100,35.776400
47,ENJ,10.492400,7.374500
52,ODE,12.349100,109.770200
53,RIF,20.359100,53.402200
70,ZRX,48.692100,291.305700
76,TOMO,19.797000,16.600400
77,GRIN,12.500000,551.103400
79,ICX,41.480100,55.260900
